
<div align="center">
  <a href="https://www.w-hs.de/maschinenbau-master-boh/">
    <img src="https://www.w-hs.de/typo3conf/ext/whs/Resources/Public/Images/Pagelayout/w-hs_pagelogo.png" 
    alt="Images" width="500" height="200">
  </a>
</div>
<br>
<h1 align="center"> Sondergebiete der Simulation</h1>
<h3 align="center"> WS 21/22 </h3>
<br>

### 1. Beschreibung des Systems: *freies, undgedämpftes inverses Pendel*
### 2. Herleitung der Bewegungsgleichungen nach Legrange
### 3. Numerische Lösung der Differentialgleichungen
### 4. Animation der Ergebnisse
<br>

### Credits:
* [zjor](https://github.com/zjor/inverted-pendulum/blob/master/python/free-cart.py)
* [lennart2810](https://github.com/lennart2810/InvertedPendulumSDS/blob/master/MKS/Inverted%20Pendulum%20Legrange.ipynb)

<div align="center">
  <a href="https://en.wikipedia.org/wiki/Inverted_pendulum">
    <img src="https://upload.wikimedia.org/wikipedia/commons/0/00/Cart-pendulum.svg" 
    alt="Images" width="250" height="100">
  </a>
</div>

## Schematische Darstellung des inversen Pendels auf einem Wagen $\rightarrow$ [Inverted pendulum](https://en.wikipedia.org/wiki/Inverted_pendulum)

> * Das System besitzt zwei Freiheitsgrade und lässt sich über die beiden Koordinaten $x$ und $\theta$ vollständig beschreiben.
> * Der Koordinatenurspung sowie das vertikale Lot liegen im Drehgelenk des Pendels
> * Der Wagen kann sich lediglich auf der x-Achse bewegen $\rightarrow y = 0$ 
> * Reibung zwischen Wagen und der Ebene sowie im Drehgelenk des Pendels werden vernachlässigt.

> * Die Pendelstange wird als masselos angenommne und hat die Länge $l$.
> * Die Masse des Wagens und die Punktmasse am Ende der Stange werden als $M$ und $m$ bezeichnet.
> * $F$ stellt eine externe Kraft zum Beschleunigen des Wagens dar ($\rightarrow$ für das freie Pendel ist $F = const = 0$)

In [ ]:
import sympy
from sympy import Symbol, symbols, Function, diff, sin, cos, Matrix, Rational, Eq, solve, lambdify
import sympy.physics.mechanics as mech
mech.init_vprinting()

Symbolische Definition der Zeit $t$, der gegebenen Größen $l, M, m, g ~und~ F$ und der Zwangsbedingung $y = 0$:

In [ ]:
t = Symbol('t')
l, M, m, g, F = symbols('l M m, g, F')

y = 0

### generalisierten Koordinaten
Definition der zeitabhängigen, generalisierten Koordinaten $x$ und $\theta$, sowie deren Ableitungen:

$x(t)$, $\dot{x}(t)$ und $\ddot{x}(t)$

In [ ]:
x = Function('x')(t)
x_d = diff(x, t)
x_dd = diff(x, t,t)

$\theta(t)$, $\dot{\theta}(t)$ und $\ddot{\theta}(t)$

In [ ]:
theta = Function('theta')(t)
theta_d = diff(theta, t)
theta_dd = diff(theta_d, t)

### Kinematik

"The branch of physics known as **kinematics** deals with using mathematical descriptions – a set of equations of motion – and definitions to explain how real-world objects move about. [...]
It doesn't regard the mass of any object in the system to describe its motion, whereas **kinetics** does." [sciencing.com](https://sciencing.com/kinetics-vs-kinematics-whats-the-difference-why-it-matters-13720229.html)


Pendelposition und -geschwindigkeit in Abhängigkeit der generalisierten Koordinaten $x$ und $\theta$

In [ ]:
x2 = x - l * sin(theta)
y2 = y + l * cos(theta)
display(Matrix([[x2], [y2]]))

x2_d = x2.diff(t)
y2_d = y2.diff(t)
display(Matrix([[x2_d], [y2_d]]))

### Kinetik
* Die kinetische Energie des Gesamtsystems berechnet sich aus den Massen und Geschwindigkeiten der Teilsysteme __Wagen__ und __Pendel__:
$$T = \sum_i \frac{m_i}{2} \cdot v_i^2$$


* Lediglich das **Pendel** trägt zur potentiellen Energie des Gesamtsystems bei: $V = m \cdot g \cdot y_2$ 

In [ ]:
T = Rational(1,2) * M * (x_d)**2 + Rational(1,2) * m * (x2_d**2 + y2_d**2)
display(T)

In [ ]:
V = M * g * y + m * g * y2 
display(V)

### Lagrange Funktion:

In [ ]:
L = T - V
display(L)

### Euler-Legrange Gleichungen:
Partielle Ableitung der Legrange Funktion nach den Koordinaten $x$ und $\theta$:

$$\frac{\partial L}{\partial x} - \frac{d}{dt}\frac{\partial L}{\partial \dot{x}} = F$$

$$\frac{\partial L}{\partial \theta} - \frac{d}{dt}\frac{\partial L}{\partial \dot{\theta}} = 0$$

In [ ]:
LE_x = diff(diff(L, x_d), t) - diff(L, x)
LE_x = Eq(LE_x, F) 

LE_theta = diff(diff(L, theta_d), t) - diff(L, theta)
LE_theta = Eq(LE_theta, 0) 

display(Matrix([[LE_x.simplify()], [LE_theta.simplify()]]))

### Lösung des Bewegungsgleichungen nach $\ddot{x}$ und $\ddot{\theta}$ mit sympy's `solve`.

In [ ]:
solutions = solve([LE_x, LE_theta], (x_dd, theta_dd), simplify=True)

#### Lösung für $\ddot{x}$:

In [ ]:
display(Eq(x_dd,solutions[x_dd]))

#### Lösung für $\ddot{\theta}$:

In [ ]:
display(Eq(theta_dd, solutions[theta_dd]))

### Gleichungssystem

Als Ergebnis bekommen wir zwei Differentialgleichungen **2. Ordnung**.

Diese müssen in DGL **1. Ordung** umgewandelt werden, damit Python diese *numerisch* Lösen kann!

Dafür werden die Variablen $v$ und $\omega$ eingeführt, um das System über 4 DGL 1. Ordnung beschreiben zu können:

1. $\dot x = v$
2. $\ddot x = \dot v$ 


3. $\dot\theta = \omega$
4. $\ddot\theta = \dot \omega$


Die **symbolischen** Ausdrücke müssen für den Solver als **numerische** Funktionen implementiert werden.

Hierfür wird sympy's `lambdify` Methode genutzt, welche die symbolischen Funktionen und deren Parameter erwartet.

* $dx/dt = v$
* $d v/dt =$ solutions[x_dd]


* $d\theta/dt = \omega$
* $d\omega/dt =$ solutions[theta_dd]

In [ ]:
dxdt = lambdify(x_d, x_d)
dvdt = lambdify((t,g,M,m,l,theta,theta_d, F), solutions[x_dd])

dthetadt = lambdify(theta_d, theta_d)
domegadt = lambdify((t,g,M,m,l,theta,theta_d, F), solutions[theta_dd])

### State-Vektor

Der Systemzustand wird über den zeitabhängigen *State-Vektor* $\vec{S} = [x, v, \theta, \omega]$ definiert.

Der Python Solver benötigt eine Funktion, die $\vec{S}$ entgegennimmt und $d\vec{S}/dt$ zurückgibt.

In [ ]:
def dSdt(S, t):
    _x, _v, _theta, _omega = S
    return [
        dxdt(_v),
        dvdt(t, g, M, m, l, _theta, _omega, F),
        dthetadt(_omega),
        domegadt(t, g, M, m, l, _theta, _omega, F)
    ]

### Symbolisch $\rightarrow$ Numerisch

Bis hierhin wurde das System ausschließlich symbolisch betrachtet. 
Der Bewegungsablauf des inversen Pendels soll nun konktret dargestellen werden.
Hierfür werden die symbolischen Variablen überschrieben:

In [ ]:
from numpy import linspace

# time
t1 = 10                          # s 
samples = 200                    # n 
t = linspace(0, t1, samples)     # s

# init
x = 0.0                          # m
v = 0.0                          # m/s
theta = 0.1                      # rad
omega = 0.0                      # rad/s
S0 = [x, v, theta, omega]        # Start-Vektor
F = 0                            # N

# const
g = 9.81                         # m/s^2
M = 5                            # kg
m = 1                            # kg
l = 1                            # m

### Numerisches Lösen der Gleichungen mit scipy's `odeint`:

Die Methode `odeint` erwartet folgende Parameter:
* die Differentialgleichungen (1. Ordnung)
* den Anfangszustand $S_0$
* den Zeitbereich $t$
* und die numerischen Konstanten als weitere Arguente

In [ ]:
from scipy.integrate import odeint

ans = odeint(dSdt, y0=S0, t=t)

x = ans.T[0]
theta = ans.T[2]

## Die numerische Berechnung der Bewegungsgleichung ist bis hierhin abgeschlossen!

### Letzendlich folgt die Darstellung der Ergebnisse:

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.patches import Rectangle
from matplotlib.gridspec import GridSpec
from matplotlib.animation import PillowWriter
%matplotlib inline
#%matplotlib notebook # --> animation im Jupyter Notebook

### Animation der Bewegungsgleichung

In [ ]:
fig = plt.figure(figsize=(16,5))
gs = GridSpec(nrows=1, ncols=2, width_ratios=[1, 2])

# Title
string = r"%s "%(' ')
fig.suptitle(string, fontsize=24)

# Pendelplot
ax1 = fig.add_subplot(gs[0, 0], autoscale_on=True, xlim=(-l*1.5, l*1.5), ylim=(-l*1.5, l*1.5)) 
ax1.get_xaxis().set_ticks([]) # enable this to hide x axis ticks
ax1.get_yaxis().set_ticks([]) # enable this to hide y axis ticks

# Wagen
cart_w, cart_h = 0.42, 0.28 # Breite, Höhe, Raddurchmesser
cart = ax1.add_patch(Rectangle((0, 0), 0, 0, linewidth=1, edgecolor='k', facecolor='g'))
wheels, = ax1.plot([], [], 'ok', markersize=8) 
ground, = ax1.plot([-1.2,1.2], [-(cart_h/2+.06),-(cart_h/2+.06)], '-k', alpha=.5, lw=2, zorder=0)

# Pendel
pendulum, = ax1.plot([], [], '-k', lw=2, marker='o') # Pendelstange als Linie darstellen
circle = plt.Circle((0,0), 0.08, fc='r', edgecolor='k', zorder=3) # Pendelmasse als Kreis darstellen
circle = ax1.add_patch(circle)

# Zeitlabel
dt = t1 / samples
time_template = 'time = %.1fs'
time_text = ax1.text(0.05, 1.02, '', transform=ax1.transAxes, fontsize=16)

# Zeitplot Wagen
ax2 = fig.add_subplot(gs[:, 1], autoscale_on=True)
ax2.set_xlabel('time [$~s~$]', fontsize=16)
ax2.plot(t,x, color="grey", linewidth=3, alpha=1, label="$x$")
ax2.legend(loc=(1.05,0.8), frameon=False, fontsize=20)
ax2.set_ylabel('position [$~m~$]', color='k', fontsize=12, labelpad=10)
ax2.grid()
x_point, = ax2.plot([], [], 'o-', color='g', markersize=12)

# Zeitplot Pendel
ax3 = ax2.twinx()
ax3.plot(t,theta, color="tab:blue", linewidth=3, alpha=1, label=r"$\theta$")
ax3.legend(loc=(1.05,0.7), frameon=False, fontsize=20)
ax3.set_ylabel('angle [$~rad~$]', fontsize=12, labelpad=14)
ax3.grid()
theta_point, = ax3.plot([], [], 'o-', color='r', markersize=10)


def init():
    cart.set_height(cart_h)
    cart.set_xy((-cart_w/2, -cart_h/2))
    cart.set_width(cart_w)
    pendulum.set_data([], [])
    wheels.set_data([],[])
    time_text.set_text('')
    return cart, pendulum, time_text

def animate(i):
    
    # Wagenposition
    cart.set_x(x[i] - cart_w/2)
    wheels.set_data([x[i]+cart_w/3, x[i]-cart_w/3],[-cart_h/2,-cart_h/2])
    
    # Pemdelmasse
    x2 = x[i] -l * sin(theta[i])
    y2 = l * cos(theta[i])
    circle.set_center((x2, y2))
    
    # Pendelstande
    thisx = [x[i], x2]
    thisy = [0, y2]
    pendulum.set_data(thisx, thisy)
    
    # Samplezeit
    time_text.set_text(time_template % (i*dt))
    
    # Zeitverlauf
    x_point.set_data(t[i], x[i])
    theta_point.set_data(t[i], theta[i])
    
    return cart, pendulum, time_text

ani = animation.FuncAnimation(fig, animate, init_func=init, frames=samples, blit=True)
ani.save('InvertedPendulum.gif', writer='pillow', fps=samples/t1, dpi=180)

In [56]:
from sympy import symbols, Matrix, pi, cos, sin 
from sympy.physics.mechanics import * 
import sympy.physics.mechanics as mech
mech.init_vprinting()
import numpy as np
# Modeling the system with Kane method
# Signals
x, theta = dynamicsymbols('x theta') 
v, omega = dynamicsymbols('x theta', 1) # level=1 --> 1. derivertive

F = dynamicsymbols('F') # external force
d = symbols('d') # damping 

# Constants
g, t = symbols('g t') 
M, m, l = symbols('M m l') 
Ic = symbols('Ic') # ka

In [57]:
# Frames and Coord. system
# Car
reference_frame = ReferenceFrame('R_f') 
X = Point('X') # cart position

X.set_vel(reference_frame, v * reference_frame.x)
Car = Particle('Car', X, M)

# Pendulum
A = reference_frame.orientnew('A','Axis' ,[theta, reference_frame.z]) 
A.set_ang_vel(reference_frame, omega * reference_frame.z)
print(type(A))

P = X.locatenew('P', l * A.x) 
P.v2pt_theory(X, reference_frame, A)
Pa = Particle('Pa', P, m)

<class 'sympy.physics.vector.frame.ReferenceFrame'>


In [58]:
I = outer(reference_frame.z, reference_frame.z) 
Inertia_tuple = (Ic * I, P)
Bp = RigidBody('Bp', P, A, m, Inertia_tuple)
Bp.potential_energy = m * g * l * sin(theta) 
Car.potential_energy = 0

In [59]:
# Forces and torques
forces = [(X, F * reference_frame.x - d * v * reference_frame.x), (P, 0 * reference_frame.y)]

In [64]:
# Lagrange operator
L = Lagrangian(reference_frame, Car, Bp)
# Lagrange model
LM = LagrangesMethod(L, [x, theta], forcelist=forces, frame=reference_frame)
LE = LM.form_lagranges_equations()
display(LE[0])
display(LE[1])
display(LE)

⎡                           ⎛                            2      ⎞             
⎢                         m⋅⎝-2⋅l⋅sin(θ)⋅θ̈ - 2⋅l⋅cos(θ)⋅θ̇  + 2⋅ẍ⎠          
⎢             M⋅ẍ + d⋅ẋ + ─────────────────────────────────────── - F       
⎢                                            2                                
⎢                                                                             
⎢                                         ⎛   2                               
⎢                                       m⋅⎝2⋅l ⋅θ̈ - 2⋅l⋅sin(θ)⋅ẍ - 2⋅l⋅cos(θ
⎢Ic⋅θ̈ + g⋅l⋅m⋅cos(θ) + l⋅m⋅cos(θ)⋅θ̇⋅ẋ + ───────────────────────────────────
⎣                                                           2                 

    ⎤
       ⎥
      ⎥
    ⎥
    ⎥
   ⎞⎥
)⋅θ̇⋅ẋ⎠⎥
───────⎥
    ⎦

In [28]:
# https://robertobucher.dti.supsi.ch/wp-content/uploads/2017/03/BookPythonForControl.pdf

# Prof. Roberto Bucher

The global reference frame is Nf (x, y) The point P is the center of mass of the pendulum. The car is moving with speed v and position C. The pole is rotating with the angle th and angular velocity w, In addition to the main coordinate frame Nf (x, y), we define a local body-fixed frame to the pendulum Npend (x1, y1).

# Add sympy.mechanics original

In [37]:
from sympy import symbols, Matrix, pi, cos, sin 
from sympy.physics.mechanics import * 
import sympy.physics.mechanics as mech
mech.init_vprinting()
import numpy as np
# Modeling the system with Kane method
# Signals
x, th = dynamicsymbols('x th') 
v, w= dynamicsymbols('x th', 1) 
F = dynamicsymbols('F')
d = symbols('d')
# Constants
m, r = symbols('m r') 
M= symbols('M')
g, t = symbols('g t') 
Ic = symbols('Ic')

In [38]:
# Frames and Coord. system
# Car
Nf = ReferenceFrame('Nf') 
C=Point('C')
C.set_vel(Nf, v*Nf.x)
Car = Particle('Car',C,M)
# Pendulum
A = Nf.orientnew('A','Axis' ,[th,Nf.z]) 
A.set_ang_vel(Nf,w*Nf.z)
P=C.locatenew('P',r*A.x) 
P.v2pt_theory(C,Nf,A)
Pa = Particle('Pa', P, m)

In [39]:
I = outer (Nf.z, Nf.z) 
Inertia_tuple = (Ic*I, P)
Bp = RigidBody('Bp', P, A, m, Inertia_tuple)
Bp.potential_energy =m*g*r*sin(th) 
Car.potential_energy = 0

In [40]:
# Forces and torques
forces = [(C,F*Nf.x - d*v*Nf.x) ,(P,0*Nf.y)]

In [41]:
# Lagrange operator
L = Lagrangian(Nf, Car, Bp)
# Lagrange model
LM= LagrangesMethod(L, [x, th], forcelist = forces , frame = Nf)
LM.form_lagranges_equations()


⎡                             ⎛                                2      ⎞       
⎢                           m⋅⎝-2⋅r⋅sin(th)⋅tḧ - 2⋅r⋅cos(th)⋅tḣ  + 2⋅ẍ⎠    
⎢               M⋅ẍ + d⋅ẋ + ─────────────────────────────────────────── - F 
⎢                                                2                            
⎢                                                                             
⎢                                             ⎛   2                           
⎢                                           m⋅⎝2⋅r ⋅tḧ - 2⋅r⋅sin(th)⋅ẍ - 2⋅r
⎢Ic⋅tḧ + g⋅m⋅r⋅cos(th) + m⋅r⋅cos(th)⋅tḣ⋅ẋ + ───────────────────────────────
⎣                                                                 2           

            ⎤
               ⎥
              ⎥
            ⎥
            ⎥
           ⎞⎥
⋅cos(th)⋅tḣ⋅ẋ⎠⎥
───────────────⎥
            ⎦

In [11]:
# Equilibrium point
eq_pt = [0.0, pi/2,0.0,0.0]
eq_dict = dict(zip([x,th,v,w], eq_pt))
# symbolically linearize about arbitrary equilibrium
MM, linear_state_matrix , linear_input_matrix , inputs =LM.linearize(q_ind = [x, th], qd_ind = [v,w])
f_p_lin = linear_state_matrix.subs(eq_dict) 
f_B_lin = linear_input_matrix.subs(eq_dict)
MM=MM.subs(eq_dict)
Atmp=MM.inv() * f_p_lin 
Btmp=MM.inv() * f_B_lin

In [12]:
display(Atmp)

⎡0           0                     1            0⎤
⎢                                                ⎥
⎢0           0                     0            1⎥
⎢                                                ⎥
⎢            2  2              ⎛        2⎞       ⎥
⎢         g⋅m ⋅r            -d⋅⎝Ic + m⋅r ⎠       ⎥
⎢0  ────────────────────  ────────────────────  0⎥
⎢                      2                     2   ⎥
⎢   Ic⋅M + Ic⋅m + M⋅m⋅r   Ic⋅M + Ic⋅m + M⋅m⋅r    ⎥
⎢                                                ⎥
⎢      g⋅m⋅r⋅(M + m)            -d⋅m⋅r           ⎥
⎢0  ────────────────────  ────────────────────  0⎥
⎢                      2                     2   ⎥
⎣   Ic⋅M + Ic⋅m + M⋅m⋅r   Ic⋅M + Ic⋅m + M⋅m⋅r    ⎦

In [13]:
display(Btmp)

⎡         0          ⎤
⎢                    ⎥
⎢         0          ⎥
⎢                    ⎥
⎢             2      ⎥
⎢     Ic + m⋅r       ⎥
⎢────────────────────⎥
⎢                   2⎥
⎢Ic⋅M + Ic⋅m + M⋅m⋅r ⎥
⎢                    ⎥
⎢        m⋅r         ⎥
⎢────────────────────⎥
⎢                   2⎥
⎣Ic⋅M + Ic⋅m + M⋅m⋅r ⎦

In [42]:
# kane original

In [44]:
from sympy import symbols, Matrix, pi 
from sympy.physics.mechanics import *
import numpy as np
# Modeling the system with Kane method
# Signals
x, th = dynamicsymbols('x th') 
v, w = dynamicsymbols('v w')
F = dynamicsymbols('F')
# Constants
d = symbols('d') # friction m, r = symbols(’m r’)
M= symbols('M')
g, t = symbols('g t') 
J = symbols('J')

In [46]:
# Frames and Coord. system
# Car − reference x,y
Nf = ReferenceFrame('Nf') 
C=Point('C')
C.set_vel(Nf, v*Nf.x)
Car = Particle('Car',C,M)
# Pendulum − reference x1, y1
Npend = Nf.orientnew('Npend','Axis' ,[th,Nf.z]) 
Npend.set_ang_vel(Nf,w*Nf.z)
P = C.locatenew('P',r*Npend.x) 
P.v2pt_theory(C,Nf,Npend)
Pa = Particle('Pa', P, m)

In [47]:
I = outer(Nf.z, Nf.z)
Inertia_tuple = (J*I, P)
Bp = RigidBody('Bp', P, Npend, m, Inertia_tuple)

In [51]:
# Forces and torques
forces = [(C,F*Nf.x-d*v*Nf.x) ,(P,-m*g*Nf.y)] 
frames = [Nf,Npend]
points = [C,P]
kindiffs = [x.diff(t)-v, th.diff(t) - w] 
particles = [Car,Bp]

In [54]:
# Model
KM= KanesMethod(Nf,q_ind=[x,th],u_ind=[v,w], kd_eqs=kindiffs)
fr , frstar = KM.kanes_equations(forces, particles )

TypeError: cannot unpack non-iterable Particle object